## Imports

In [31]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd

from pandas_geojson import to_geojson, write_geojson
from geopy.geocoders import GoogleV3
from textwrap import shorten

## Read-in

In [32]:
df = pd.read_csv('construction_loan_roundup - Copy of Sheet1.csv')

In [33]:
df.columns

Index(['Amount', 'Project', 'Developers_borrowers', 'Lenders', 'Property type',
       'Date loan closed', 'Address ', 'County ', 'story_link'],
      dtype='object')

## Google Maps API Geolocater Setup

In [34]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [35]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['Address '].apply(geocode)

In [36]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Article Link Formatter

In [37]:
df["short_description"] = df["Developers_borrowers"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Developers_borrowers'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/07...
1    <a href="https://therealdeal.com/miami/2023/01...
2    <a href="https://therealdeal.com/miami/2022/12...
3    <a href="https://therealdeal.com/miami/2022/04...
4    <a href="https://therealdeal.com/miami/2023/01...
5    <a href="https://therealdeal.com/miami/2022/08...
6    <a href="https://therealdeal.com/miami/2023/01...
7    <a href="https://therealdeal.com/miami/2022/04...
8    <a href="https://therealdeal.com/miami/2022/02...
9    <a href="https://therealdeal.com/miami/2022/09...
Name: description_link, dtype: object

## Write to GeoJSON

In [38]:
df.columns

Index(['Amount', 'Project', 'Developers_borrowers', 'Lenders', 'Property type',
       'Date loan closed', 'Address ', 'County ', 'story_link', 'geocoded',
       'lat', 'lon', 'short_description', 'remaining_desc',
       'description_link'],
      dtype='object')

In [39]:
geo_json = to_geojson(df=df, lat='lat', lon='lon',
                     properties=['Amount','Project','Developers_borrowers','Lenders','Property type','Date loan closed','Address ','County ','story_link','geocoded','short_description','remaining_desc','description_link'])
write_geojson(geo_json, filename='SoFla_top_mortgages.geojson', indent=4)

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/SoFla_top_10_mortgages_scrollytelling'

In [41]:
print(base_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_top_10_mortgages_scrollytelling
